In [2]:
import pickle


In [21]:
!pip install inflect

In [64]:
with open('bad.pickle', 'rb') as f:
    bad_android = pickle.load(f)
with open('features.pickle', 'rb') as f:
    feature_android = pickle.load(f)
with open('features_ios.pickle', 'rb') as f:
    features_ios = pickle.load(f)



In [68]:
import re, string, unicodedata
import nltk
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
import numpy as np
import pandas as pd

def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", sample)

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

def preprocess(sample):
    soup = BeautifulSoup(sample)
    sample = soup.get_text()

    sample = remove_URL(sample)
    # Tokenize
    words = nltk.word_tokenize(sample)

    # Normalize
    words = normalize(words)
    words = " ".join(words)
    return words


clear_text = []
for i, bundle in enumerate(features_ios):
    try:
        if not pd.isna(features_ios[bundle]["name"]) and features_ios[bundle]["name"] != "nan":
            sample = preprocess(features_ios[bundle]["name"])
            clear_text.append(sample)
        else:
            clear_text.append(" ")

        if i % 100 == 0:
            print(i/len(features_ios))
    except:
        print("Error")
        clear_text.append(" ")
        continue


# clear_text = []
# for i, bundle in enumerate(feature_android):
#     try:
#         if not pd.isna(feature_android[bundle]["name"]) and feature_android[bundle]["name"] != "nan":
#             sample = preprocess(feature_android[bundle]["name"])
#             clear_text.append(sample)
#         if i % 100 == 0:
#             print(i/len(feature_android))
#     except:
#         print("Error")
#         clear_text.append(" ")
#         continue

0.0
0.0076546233925290875
0.015309246785058175
0.02296387017758726
0.03061849357011635
0.03827311696264544
0.04592774035517452
0.053582363747703615
0.0612369871402327
0.06889161053276179
0.07654623392529088
0.08420085731781997
0.09185548071034905
0.09951010410287814
0.10716472749540723
0.11481935088793631
0.1224739742804654
0.13012859767299448
0.13778322106552357
0.14543784445805266
0.15309246785058175
0.16074709124311085
0.16840171463563994
0.176056338028169
0.1837109614206981
0.19136558481322719
0.19902020820575628
0.20667483159828537
0.21432945499081446
0.22198407838334355
0.22963870177587262
0.2372933251684017
0.2449479485609308
0.25260257195345986
0.26025719534598896
0.26791181873851805
0.27556644213104714
0.28322106552357623
0.2908756889161053
0.2985303123086344
0.3061849357011635
0.3138395590936926
0.3214941824862217
0.3291488058787508
0.3368034292712799
0.34445805266380897
0.352112676056338
0.3597672994488671
0.3674219228413962
0.3750765462339253
0.38273116962645437
0.390385793

In [69]:
import sys

sys.path.append('..')
from text2vec import SBert

sbert_model = SBert('paraphrase-multilingual-MiniLM-L12-v2')
vectorize = sbert_model.encode(clear_text)

In [70]:
for i, bundle in enumerate(features_ios):
    features_ios[bundle]["vectorized"] = np.mean(vectorize[i]) 

In [67]:
for i, bundle in enumerate(features_ios):
    try:
        if not pd.isna(features_ios[bundle]["name"]) and features_ios[bundle]["name"] != "nan":
            features_ios[bundle]["vectorized"] = np.mean(vectorize[i]) 
        if i % 100 == 0:
            print(i/len(features_ios))
    except:
        features_ios[bundle]["vectorized"] = np.mean(vectorize[i]) 
        continue

0.0
0.0076546233925290875
0.015309246785058175
0.02296387017758726
0.03061849357011635
0.03827311696264544
0.04592774035517452
0.053582363747703615
0.0612369871402327
0.06889161053276179
0.07654623392529088
0.08420085731781997
0.09185548071034905


IndexError: index 1218 is out of bounds for axis 0 with size 1206

In [63]:
with open('feature_android_vectorized.pickle', 'wb') as f:
    pickle.dump(feature_android, f)


In [72]:
with open('features_ios_vectorized.pickle', 'wb') as f:
    pickle.dump(features_ios, f)
